In [1]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np
from loadfft import getData

srng = RandomStreams()

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def rectify(X):
    return T.maximum(X, 0.)

def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

def dropout(X, p=0.):
    if p > 0:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
        X /= retain_prob
    return X

def model(X, w_h, w_h2,w_h3, w_o, p_drop_input, p_drop_hidden):
    X = dropout(X, p_drop_input)
    h = rectify(T.dot(X, w_h))

    h = dropout(h, p_drop_hidden)
    h2 = rectify(T.dot(h, w_h2))

    h2 = dropout(h2, p_drop_hidden)
    h3= rectify(T.dot(h2, w_h3))

    h3 = dropout(h3, p_drop_hidden)
    py_x = softmax(T.dot(h3, w_o))
    return h, h2,h3, py_x

trX,trY,teX,teY = getData(20)

X = T.fmatrix()
Y = T.fmatrix()

w_h = init_weights((93, 625))
w_h2 = init_weights((625, 625))
w_h3=init_weights((625, 625))
w_o = init_weights((625, 4))

noise_h, noise_h2,noise_h3, noise_py_x = model(X, w_h, w_h2,w_h3, w_o, 0.2, 0.5)
h, h2,h3, py_x = model(X, w_h, w_h2,w_h3, w_o, 0., 0.)
y_x = T.argmax(py_x, axis=1)

cost = T.mean(T.nnet.categorical_crossentropy(noise_py_x, Y))
params = [w_h, w_h2,w_h3, w_o]
updates = RMSprop(cost, params, lr=0.001)

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True)


Couldn't import dot_parser, loading of dot files will not be possible.
111
155
26
37


/home/notebook/.venv/lib/python2.7/site-packages/numpy/core/numeric.py:462: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [2]:
print(len(trX))
for i in range(100):
    for start, end in zip(range(0, len(trX), 10), range(10, len(trX), 10)):
        cost = train(trX[start:end], trY[start:end])
    print i,np.mean(np.argmax(trY, axis=1) == predict(trX)),np.mean(np.argmax(teY, axis=1) == predict(teX))
    if i==99:
        print("predict trX")
        print(predict(trX))
        print("predict teX")
        print(predict(teX))
        print("teY")
        print(np.argmax(teY, axis=1))

327
0 0.782874617737 0.607594936709
1 0.770642201835 0.582278481013
2 0.85626911315 0.651898734177
3 0.85626911315 0.550632911392
4 0.889908256881 0.651898734177
5 0.868501529052 0.607594936709
6 0.889908256881 0.70253164557
7 0.905198776758 0.651898734177
8 0.917431192661 0.70253164557
9 0.908256880734 0.588607594937
10 0.926605504587 0.613924050633
11 0.917431192661 0.620253164557
12 0.935779816514 0.601265822785
13 0.923547400612 0.563291139241
14 0.929663608563 0.613924050633
15 0.954128440367 0.639240506329
16 0.938837920489 0.607594936709
17 0.94495412844 0.613924050633
18 0.969418960245 0.626582278481
19 0.963302752294 0.626582278481
20 0.960244648318 0.613924050633
21 0.963302752294 0.626582278481
22 0.963302752294 0.632911392405
23 0.966360856269 0.626582278481
24 0.966360856269 0.645569620253
25 0.975535168196 0.639240506329
26 0.975535168196 0.613924050633
27 0.969418960245 0.658227848101
28 0.960244648318 0.645569620253
29 0.966360856269 0.620253164557
30 0.97247706422 0.65

In [6]:
range(1,3)


[1, 2]

In [12]:
import numpy as np
np.array([])


array([], dtype=float64)

In [13]:
++1


1

In [14]:
a=1

In [15]:
++a

1

In [16]:
++a

1

In [17]:
a+=1

In [18]:
a


2